# Custom RAG Retriever: Superlinked + LlamaIndex Integration

This notebook demonstrates how to build a custom retriever by combining **Superlinked** (vector compute engine) with **LlamaIndex**'s retrieval framework for a Steam game recommendation system.

## Key Features:
- Custom domain-specific retriever
- Semantic similarity using sentence transformers
- Multi-field indexing and advanced query logic
- Real-time recommendations with sub-millisecond latency

## Architecture:
```
User Query → Custom Retriever → Superlinked Vector Search → LlamaIndex Nodes → LLM Response
```

## 1. Install Dependencies

We need to install Superlinked and LlamaIndex packages for our custom retriever implementation.

In [ ]:
%%capture
%pip install superlinked==23.7.0 llama-index llama-index-llms-openai pandas sentence-transformers huggingface_hub numpy

## 2. Downloading the data and setting up Hugging face

In [ ]:
import pandas as pd

!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1jgjcnxrjAikMrzHKKk5KsMf4lYaGWr9j' -O data.csv

--2025-05-30 11:35:48--  https://drive.google.com/uc?export=download&id=1jgjcnxrjAikMrzHKKk5KsMf4lYaGWr9j
Resolving drive.google.com (drive.google.com)... 173.194.194.101, 173.194.194.113, 173.194.194.138, ...
Connecting to drive.google.com (drive.google.com)|173.194.194.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1jgjcnxrjAikMrzHKKk5KsMf4lYaGWr9j&export=download [following]
--2025-05-30 11:35:48--  https://drive.usercontent.google.com/download?id=1jgjcnxrjAikMrzHKKk5KsMf4lYaGWr9j&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.69.132, 2607:f8b0:4001:c6e::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.69.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1696215 (1.6M) [application/octet-stream]
Saving to: ‘data.csv’

data.csv            100%[===================>]   1.62M  --.-KB/s    

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `custom-retriever` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `custom-

## 2. Import Required Libraries

Setting up all necessary imports for our Retrieval pipeline:
- **LlamaIndex Core**: Provides retrieval abstraction layer
- **Superlinked Framework**: Handles vector computation and semantic search  

In [ ]:
import time
import logging
import pandas as pd
from typing import List
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore, QueryBundle, TextNode
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
import superlinked.framework as sl

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


## 3. Understanding LlamaIndex Custom Retrievers

LlamaIndex provides an abstract `BaseRetriever` class that serves as the foundation for all retrieval operations. The power lies in its simplicity—any custom retriever only needs to implement one core method:

### The Retrieval Protocol:
1. **Input**: `QueryBundle` - Contains user query and optional metadata
2. **Output**: `List[NodeWithScore]` - Retrieved content chunks with relevance scores
3. **Processing**: Backend-agnostic - Use any search strategy you want

This abstraction allows you to:
- Combine multiple search strategies
- Run A/B tests easily
- Plug into any agent or tool system

## 4. Custom Superlinked Steam Games Retriever

Our custom retriever inherits from `BaseRetriever` and integrates Superlinked's vector computation capabilities. Key components:

- **Schema Definition**: Type-safe data structure for games
- **Text Similarity Space**: Semantic search using sentence transformers
- **Combined Text Field**: Concatenates multiple fields for richer semantic matching
- **In-Memory Execution**: Lightning-fast queries for real-time recommendations

In [ ]:
class SuperlinkedSteamGamesRetriever(BaseRetriever):
    """A custom LlamaIndex retriever using Superlinked for Steam games data."""

    def __init__(self, csv_file: str, top_k: int = 10):
        """
        Initialize the retriever with a CSV file path and top_k parameter.

        Args:
            csv_file (str): Path to games_data.csv
            top_k (int): Number of results to return (default: 10)
        """
        self.top_k = top_k
        # Load the dataset and ensure all required columns are present
        self.df = pd.read_csv(csv_file)
        print(f"Loaded dataset with {len(self.df)} games")
        print("DataFrame Columns:", list(self.df.columns))

        required_columns = [
            'game_number', 'name', 'desc_snippet', 'game_details', 'languages',
            'genre', 'game_description', 'original_price', 'discount_price'
        ]
        for col in required_columns:
            if col not in self.df.columns:
                raise ValueError(f"Missing required column: {col}")

        # Combine relevant columns into a single field for text similarity
        self.df['combined_text'] = (
            self.df['name'].astype(str) + " " +
            self.df['desc_snippet'].astype(str) + " " +
            self.df['genre'].astype(str) + " " +
            self.df['game_details'].astype(str) + " " +
            self.df['game_description'].astype(str)
        )

        self._setup_superlinked()

    def _setup_superlinked(self):
        """Set up Superlinked schema, space, index, and executor."""
        # Define schema
        class GameSchema(sl.Schema):
            game_number: sl.IdField
            name: sl.String
            desc_snippet: sl.String
            game_details: sl.String
            languages: sl.String
            genre: sl.String
            game_description: sl.String
            original_price: sl.Float
            discount_price: sl.Float
            combined_text: sl.String  # New field for combined text

        self.game = GameSchema()

        # Create text similarity space using the combined_text field
        self.text_space = sl.TextSimilaritySpace(
            text=self.game.combined_text,
            model="sentence-transformers/all-mpnet-base-v2"
        )

        # Create index
        self.index = sl.Index([self.text_space])

        # Map DataFrame columns to schema
        parser = sl.DataFrameParser(
            self.game,
            mapping={
                self.game.game_number: "game_number",
                self.game.name: "name",
                self.game.desc_snippet: "desc_snippet",
                self.game.game_details: "game_details",
                self.game.languages: "languages",
                self.game.genre: "genre",
                self.game.game_description: "game_description",
                self.game.original_price: "original_price",
                self.game.discount_price: "discount_price",
                self.game.combined_text: "combined_text"
            }
        )

        # Set up in-memory source and executor
        source = sl.InMemorySource(self.game, parser=parser)
        self.executor = sl.InMemoryExecutor(sources=[source], indices=[self.index])
        self.app = self.executor.run()

        # Load data
        source.put([self.df])
        print(f"Initialized Superlinked retriever with {len(self.df)} games")

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """
        Retrieve top-k games based on the query string.

        Args:
            query_bundle (QueryBundle): Contains the query string

        Returns:
            List[NodeWithScore]: List of retrieved games with scores
        """
        query_text = query_bundle.query_str

        # Define Superlinked query with explicit field selection
        query = (
            sl.Query(self.index)
            .find(self.game)
            .similar(self.text_space, query_text)
            .select([
                self.game.game_number,
                self.game.name,
                self.game.desc_snippet,
                self.game.game_details,
                self.game.languages,
                self.game.genre,
                self.game.game_description,
                self.game.original_price,
                self.game.discount_price
            ])
            .limit(self.top_k)
        )

        # Execute query
        result = self.app.query(query)
        df_result = sl.PandasConverter.to_pandas(result)

        # Convert results to NodeWithScore objects
        nodes_with_scores = []
        for i, row in df_result.iterrows():
            text = f"{row['name']}: {row['desc_snippet']}"
            metadata = {
                "game_number": row["id"],
                "name": row["name"],
                "desc_snippet": row["desc_snippet"],
                "game_details": row["game_details"],
                "languages": row["languages"],
                "genre": row["genre"],
                "game_description": row["game_description"],
                "original_price": row["original_price"],
                "discount_price": row["discount_price"]
            }
            score = 1.0 - (i / self.top_k)
            node = TextNode(text=text, metadata=metadata)
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

print("✅ SuperlinkedSteamGamesRetriever class defined successfully!")

✅ SuperlinkedSteamGamesRetriever class defined successfully!


## 5. Enhanced Retriever with Monitoring

Adding monitoring capabilities to track query performance and results. This wrapper logs:
- Query processing latency
- Number of results returned
- Query text for debugging
- Error handling for failed retrievals

This is essential for production systems where you need to monitor retriever performance.

In [ ]:
class MonitoredRetriever(SuperlinkedSteamGamesRetriever):
    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        start_time = time.time()

        try:
            results = super()._retrieve(query_bundle)
            latency = time.time() - start_time

            print(f"Query: '{query_bundle.query_str}'")
            print(f"Latency: {latency:.3f}s")
            print(f"Results: {len(results)}")
            print("-" * 50)

            return results
        except Exception as e:
            print(f"Retrieval failed: {str(e)}")
            raise

print("✅ MonitoredRetriever class defined successfully!")

✅ MonitoredRetriever class defined successfully!


## 6. Configure OpenAI API for LLM initialization

Set up your OpenAI API key for the LLM component. Replace `"your-openai-api-key-here"` with your actual API key.

In [ ]:
import os

# Replace with your actual OpenAI API key
api_key = "your-openai-api-key-here"

try:
    if not api_key:
        raise ValueError("OpenAI API key is not set.")
    Settings.llm = OpenAI(api_key=api_key, model="gpt-3.5-turbo")
    print("✅ OpenAI API configured successfully!")
except ValueError as e:
    print(f"Error: {e}")
    print("Please replace 'your-openai-api-key-here' with your actual OpenAI API key.")
except Exception as e:
    print(f"An unexpected error occurred during OpenAI configuration: {e}")

✅ OpenAI API configured successfully!


## 7. Initialize Custom Retriever

Create an instance of our monitored retriever and test it with sample queries. This demonstrates:

- **Retriever Initialization**: Loading the dataset and setting up Superlinked
- **Semantic Search**: Testing queries like "party game" and "magic game with spells"
- **Performance Monitoring**: Tracking query latency and result count
- **Result Quality**: Examining retrieved games and relevance scores

In [ ]:
print("Initializing Custom Retriever...")
retriever = MonitoredRetriever("data.csv", top_k=3)

print("\n" + "="*60)
print("TESTING RETRIEVAL FUNCTIONALITY")
print("="*60)

# Test query 1: Party games
print("\nTesting Query: 'party game'")
results = retriever.retrieve("party game")
for i, res in enumerate(results, 1):
    print(f"   {i}. {res.node.text} (Score: {res.score:.2f})")

# Test query 2: Magic games
print("\nTesting Query: 'magic game with spells'")
results = retriever.retrieve("magic game with spells")
for i, res in enumerate(results, 1):
    print(f"   {i}. {res.node.text} (Score: {res.score:.2f})")

# Test query 3: Strategy games
print("\nTesting Query: 'strategy space exploration'")
results = retriever.retrieve("strategy space exploration")
for i, res in enumerate(results, 1):
    print(f"   {i}. {res.node.text} (Score: {res.score:.2f})")

print("\n✅ Retriever testing completed!")

Initializing Custom Retriever...
Loaded dataset with 1000 games
DataFrame Columns: ['game_number', 'name', 'desc_snippet', 'game_details', 'languages', 'genre', 'game_description', 'original_price', 'discount_price']


Fetching 28 files:   0%|          | 0/28 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


11:53:56 huggingface_hub.file_download WARNING   Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

12:03:37 superlinked.framework.query.query_dag_evaluator INFO   evaluated query
12:03:37 superlinked.framework.dsl.executor.query.query_executor INFO   executed query
Query: 'party game'
Latency: 0.376s
Results: 3
--------------------------------------------------
   1. snake party: snake party is the ultimate party game enabling up to six players to quickly arrange a snake race on the same computer! they can choose their own colors, type in their names and the game party is ready to go. (Score: 1.00)
   2. battleblock theater®: welcome to battleblock theater! you’ve got no where to go but up...on stage. play single player or co-op to free your friends and save hatty hattington! jump, solve and battle your way through a mysterious theater inhabited by highly technological felines. (Score: 0.67)
   3. xmas shooting - scramble!!: help aru save christmas in this humoristic, action-packed shooter set in the universe of qp shooting - dangerous!! (Score: 0.33)

Testing Query: 'magic game wit

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

12:03:37 superlinked.framework.query.query_dag_evaluator INFO   evaluated query
12:03:37 superlinked.framework.dsl.executor.query.query_executor INFO   executed query
Query: 'magic game with spells'
Latency: 0.087s
Results: 3
--------------------------------------------------
   1. lords of magic: special edition: urak, once peaceful, is now a land of darkness, chaos and death. the people yearn for a champion to drive away the forces of evil. that champion is you. (Score: 1.00)
   2. merlin soccer: a simple arcade game in which you have to play magic football. control the balls and score them into the goal, which is guarded by a terrible troll. (Score: 0.67)
   3. magical battle festa: teos, an agency of holy mages, holds a tournament to find the best magic users in the world to take on a dark threat looming on the horizon. choose your mage and take to the arena in this fast-paced competitive fighting game that mixes troop formations and fast action. (Score: 0.33)

Testing Query: 'stra

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

12:03:38 superlinked.framework.query.query_dag_evaluator INFO   evaluated query
12:03:38 superlinked.framework.dsl.executor.query.query_executor INFO   executed query
Query: 'strategy space exploration'
Latency: 0.079s
Results: 3
--------------------------------------------------
   1. stellaris: explore and discover a spectacular and ever-changing universe! paradox development studio, makers of the europa universalis and crusader kings series, and publishers of the best-selling cities: skylines, presents stellaris, advancing the genre of grand strategy to the very edges of the universe. (Score: 1.00)
   2. space empires v: space empires v is the latest edition in the space empires series. this new chapter completely updates the ui and takes the player into a real-time rendered 3d universe. watch space battles played out in glorious detail and realistic effects. expand, explore, exploit, and exterminate in a huge living breathing galaxy. (Score: 0.67)
   3. space engineers: space engin

## 8. Complete Retrieval Pipeline Integration

Now we combine our custom retriever with LlamaIndex's query engine to create a full RAG system. This integration:

- **Retriever**: Our custom Superlinked-powered semantic search
- **Response Synthesizer**: LlamaIndex component that formats retrieved content
- **Query Engine**: Orchestrates retrieval + LLM response generation
- **LLM**: OpenAI model that generates final responses based on retrieved context

The pipeline flow: `User Query → Semantic Retrieval → Context Assembly → LLM Response`

In [ ]:
print("Setting up complete Retrieval pipeline...")

# Create response synthesizer and query engine
response_synthesizer = get_response_synthesizer()
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer
)

print("✅ RAG pipeline configured successfully!")

print("\n" + "="*60)
print("FULL RAG PIPELINE DEMONSTRATION")
print("="*60)

# Test queries with full RAG responses
test_queries = [
    "I want to find a magic game with spells and wizards",
    "Recommend a fun party game for friends",
    "I'm looking for a strategic sci-fi game",
    "What's a good cooperative game for teamwork?"
]

for i, query in enumerate(test_queries, 1):
    print(f"\nQuery {i}: '{query}'")
    print("-" * 50)

    response = query_engine.query(query)
    print(f"Response: {response}")
    print("\n" + "="*50)

Setting up complete Retrieval pipeline...
✅ RAG pipeline configured successfully!

FULL RAG PIPELINE DEMONSTRATION

Query 1: 'I want to find a magic game with spells and wizards'
--------------------------------------------------
12:13:20 superlinked.framework.dsl.query.query_clause.select_clause INFO   The id field is automatically included - no need to specify it in the select clause.
12:13:20 superlinked.framework.query.query_dag_evaluator INFO   evaluated query
12:13:20 superlinked.framework.dsl.executor.query.query_executor INFO   executed query
Query: 'I want to find a magic game with spells and wizards'
Latency: 0.018s
Results: 3
--------------------------------------------------
12:13:21 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Response: Lords of Magic: Special Edition blends classic fantasy role-playing with real-time strategy combat, allowing players to wield fantastic sorcery, explore lands, develop abilities, and form powe

## 9. Performance Analysis and Insights

Let's analyze the performance characteristics of our custom retriever and compare different query types.

**Key Metrics:**
- **Latency**: Time taken for retrieval operations
- **Relevance**: Quality of retrieved results
- **Semantic Understanding**: How well it handles natural language queries
- **Scalability**: Performance with different dataset sizes

In [ ]:
import time

print("PERFORMANCE ANALYSIS")
print("="*50)

# Test different query complexities
performance_queries = [
    ("Simple", "magic"),
    ("Medium", "party game for friends"),
    ("Complex", "strategic cooperative space exploration with teamwork"),
    ("Specific", "medieval battle tactics with historical accuracy"),
    ("Abstract", "fun social experience with puzzles and challenges")
]

performance_results = []

for complexity, query in performance_queries:
    start_time = time.time()
    results = retriever.retrieve(query)
    end_time = time.time()

    latency = end_time - start_time
    performance_results.append({
        'complexity': complexity,
        'query': query,
        'latency': latency,
        'results_count': len(results)
    })

    print(f"{complexity:8} | {latency:.4f}s | {len(results)} results | '{query}'")

print(f"\nAverage latency: {sum(r['latency'] for r in performance_results) / len(performance_results):.4f}s")
print("✅ Performance analysis completed!")

PERFORMANCE ANALYSIS
12:13:28 superlinked.framework.dsl.query.query_clause.select_clause INFO   The id field is automatically included - no need to specify it in the select clause.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

12:13:28 superlinked.framework.query.query_dag_evaluator INFO   evaluated query
12:13:28 superlinked.framework.dsl.executor.query.query_executor INFO   executed query
Query: 'magic'
Latency: 0.104s
Results: 3
--------------------------------------------------
Simple   | 0.1045s | 3 results | 'magic'
12:13:28 superlinked.framework.dsl.query.query_clause.select_clause INFO   The id field is automatically included - no need to specify it in the select clause.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

12:13:28 superlinked.framework.query.query_dag_evaluator INFO   evaluated query
12:13:28 superlinked.framework.dsl.executor.query.query_executor INFO   executed query
Query: 'party game for friends'
Latency: 0.091s
Results: 3
--------------------------------------------------
Medium   | 0.0912s | 3 results | 'party game for friends'
12:13:28 superlinked.framework.dsl.query.query_clause.select_clause INFO   The id field is automatically included - no need to specify it in the select clause.


...

12:13:28 superlinked.framework.query.query_dag_evaluator INFO   evaluated query
12:13:28 superlinked.framework.dsl.executor.query.query_executor INFO   executed query
Query: 'strategic cooperative space exploration with teamwork'
Latency: 0.104s
Results: 3
--------------------------------------------------
Complex  | 0.1046s | 3 results | 'strategic cooperative space exploration with teamwork'
12:13:28 superlinked.framework.dsl.query.query_clause.select_clause INFO   The id field is automatically included - no need to specify it in the select clause.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

12:13:28 superlinked.framework.query.query_dag_evaluator INFO   evaluated query
12:13:28 superlinked.framework.dsl.executor.query.query_executor INFO   executed query
Query: 'medieval battle tactics with historical accuracy'
Latency: 0.092s
Results: 3
--------------------------------------------------
Specific | 0.0924s | 3 results | 'medieval battle tactics with historical accuracy'
12:13:28 superlinked.framework.dsl.query.query_clause.select_clause INFO   The id field is automatically included - no need to specify it in the select clause.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

12:13:28 superlinked.framework.query.query_dag_evaluator INFO   evaluated query
12:13:28 superlinked.framework.dsl.executor.query.query_executor INFO   executed query
Query: 'fun social experience with puzzles and challenges'
Latency: 0.108s
Results: 3
--------------------------------------------------
Abstract | 0.1093s | 3 results | 'fun social experience with puzzles and challenges'

Average latency: 0.1004s
✅ Performance analysis completed!
